In [107]:
from dataclasses import dataclass
import pandas as pd
import os 
import numpy as np
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import cv2
from transformers import AutoImageProcessor
import torch
from torchvision import transforms as T
from PIL import Image

from datasets import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from PIL import Image
from ultralytics import YOLO
import os
import cv2
from tqdm import tqdm

In [108]:
class CustomPeopleDataset(Dataset):
    def __init__(self, data_table_path, label_table_path, processor, base_dir='./'):
        self._data = pd.read_csv(data_table_path, sep=';')
        labels_table = pd.read_csv(label_table_path)

        self.labels_map = {label: i for i, label in enumerate(labels_table['name'])}
        self.base_dir = base_dir
        self.processor = processor

    def __len__(self):
        return self._data.shape[0]

    def __getitem__(self, idx):
        image_path = f"{self.base_dir}{self._data['relative_path'][idx]}/{self._data['image_name'][idx]}"
        
        image = Image.open(image_path).convert('RGB')
        image_tensor = self.processor(image)
        image.close()
        
        label = self.labels_map[self._data['label'][idx]]

        return image_tensor, label
    
    def __getitems__(self, idxs):
        return [self.__getitem__(idx) for idx in idxs]
        

def custom_collate(data):

    images = torch.cat([torch.unsqueeze(item[0], 0) for item in data], 0)
    labels = torch.tensor([item[1] for item in data])

    return {
        "images": images, 
        "labels": labels
    }

normalize = T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
TRANSFORM_MYCNN = T.Compose([
    T.Resize(112),
    T.ToTensor(),
    #normalize
    ])

In [109]:
data_table_path = '/home/dzigen/Desktop/ITMO/sem1/ImgGen/Лабы/task2/data/images_info.csv'
label_table_path = '/home/dzigen/Desktop/ITMO/sem1/ImgGen/Лабы/task2/data/lfw/people.csv'

In [110]:
train_dataset = CustomPeopleDataset(data_table_path,label_table_path, TRANSFORM_MYCNN)

### Подготовка тренировочных изображений (детекцию,вырезание,ресайз)

Source: https://colab.research.google.com/drive/1L346M4dowPtaEyKDNxl5OwJTsswfld62#scrollTo=5f0oJwg-26Tk

In [111]:
detector_model = YOLO("../yolov8n-face.pt")

In [112]:
IMG_SIZE = (112, 112)

def get_central_of_face(face_info):
    left_eye, right_eye, nose = face_info['key_points'][:3]
    center_x = (left_eye[0] + right_eye[0] + nose[0]) // 3
    center_y = (left_eye[1] + right_eye[1] + nose[1]) // 3
    face_center_point = center_x, center_y
    return face_center_point

def find_most_central_face(img, faces):
    image_height, image_width, C = img.shape

    min_distance = float('inf')
    most_central_face = None
    most_c_x, most_c_y = None, None

    for face_info in faces:
        center_x, center_y = get_central_of_face(face_info)
        distance = ((center_x - image_width/2)**2 + (center_y - image_height/2)**2)**0.5

        if distance < min_distance:
            min_distance = distance
            most_central_face = face_info
            most_c_x, most_c_y = center_x, center_y

    return most_central_face, most_c_x, most_c_y

def crop_with_padding(image, new_bbox):
    return np.array(Image.fromarray(image).crop(new_bbox))

def preprocess_image(image_path):
    res = detector_model.predict(source=image_path,
                            show=False,
                            save=False,
                            conf=0.4,
                            save_txt=False,
                            save_crop=False,
                            verbose=False,
                            )[0]
    faces = []
    for ind in range(len(res.boxes)):
        box_points = res.boxes.xyxy[ind].cpu().numpy().astype(int)
        keypoints = res.keypoints[ind].data.cpu().numpy()[0][:, :2].astype(int)
        faces.append(
            {
                "conf": res.boxes.conf[ind].cpu().item(),
                "box_points": box_points.tolist(),
                "key_points": keypoints.tolist(),
            }
        )

    if len(faces) == 0:
        print(f"Error: found no faces on image: {image_path}!!!")
        return None

    img = cv2.imread(image_path)
    face_info, center_x, center_y = find_most_central_face(img, faces)

    box_points = face_info['box_points']

    max_sz = max(
            abs(box_points[0] - center_x),
            abs(box_points[1] - center_y),
            abs(box_points[2] - center_x),
            abs(box_points[3] - center_y),
        )

    new_bbox = (center_x - max_sz, center_y - max_sz,
                center_x + max_sz, center_y + max_sz)
    croped_img = crop_with_padding(img, new_bbox)
    preprocessed_image = cv2.resize(croped_img, IMG_SIZE)
    return preprocessed_image

In [113]:
LFW_yolo_root = '../data/LFW-yolo'
dataset_root = '/home/dzigen/Desktop/ITMO/sem1/ImgGen/Лабы/task2/data/lfw/lfw-deepfunneled/lfw-deepfunneled'
os.makedirs(LFW_yolo_root, exist_ok=True)

for person_folder in tqdm(os.listdir(dataset_root)):
    person_folder_path = os.path.join(dataset_root, person_folder)

    output_person_folder_path = os.path.join(LFW_yolo_root, person_folder)
    os.makedirs(output_person_folder_path, exist_ok=True)

    for image_file in os.listdir(person_folder_path):
        # print(image_file)
        image_path = os.path.join(person_folder_path, image_file)

        preprocessed_image = preprocess_image(image_path)
        output_image_path = os.path.join(output_person_folder_path, image_file)
        cv2.imwrite(output_image_path, preprocessed_image)

100%|██████████| 5749/5749 [05:03<00:00, 18.94it/s]
